In [1]:
# 初始化环境变量
import os
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。
# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
_ = load_dotenv(find_dotenv())


def get_openai_api_key() -> str:
    return os.environ.get("OPENAI_API_KEY")


def get_openai_api_base() -> str:
    return os.environ.get("OPENAI_API_BASE")


def get_openai_model_name() -> str:
    return os.environ.get("OPENAI_MODEL_NAME")


print(f"openai_api_key: {get_openai_api_key()}")
print(f"openai_api_base: {get_openai_api_base()}")
print(f"openai_model_name: {get_openai_model_name()}")

openai_api_key: cannot be empty
openai_api_base: http://10.191.193.3:11434/v1
openai_model_name: qwen2.5:14b


In [2]:
# 创建一个Swarm对象
from openai import OpenAI
from swarm import Swarm, Agent

# 实例化客户端
swarm_client = Swarm(
    client=OpenAI(api_key=get_openai_api_key(), base_url=get_openai_api_base())
)


def instructions(context_variables: dict):
    name = context_variables.get("name", "User")
    return f"You are a helpful weather assistant. Greet the user by name ({name})."


def get_weather(location: str) -> str:
    return "{'temp':67, 'unit':'F'}"


weather_agent = Agent(
    name="Weather Agent",
    model=get_openai_model_name(),
    instructions=instructions,
    functions=[get_weather],
)

context_variables = {"name": "James"}

messages = [{"role": "user", "content": "What's the weather in NYC?"}]

response = swarm_client.run(
    messages=messages,
    agent=weather_agent,
    context_variables=context_variables,
    debug=True,
)

print(response.messages[-1]["content"])

[2024-12-23 14:39:11] Getting chat completion for...: [{'role': 'system', 'content': 'You are a helpful weather assistant. Greet the user by name (James).'}, {'role': 'user', 'content': "What's the weather in NYC?"}]
[2024-12-23 14:39:41] Received completion: ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_nkvotfsm', function=Function(arguments='{"location":"NYC"}', name='get_weather'), type='function'), ChatCompletionMessageToolCall(id='call_kdg805m6', function=Function(arguments='{"location":"New York City"}', name='get_weather'), type='function')])
[2024-12-23 14:39:41] Processing tool call: get_weather with arguments {'location': 'NYC'}
[2024-12-23 14:39:41] Processing tool call: get_weather with arguments {'location': 'New York City'}
[2024-12-23 14:39:41] Getting chat completion for...: [{'role': 'system', 'content': 'You are a helpful weather assistant. Greet the user by name (Ja